In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/random-forest/random_forest_model.pkl
/kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl
/kaggle/input/inputs/xgb_aes_model.pkl
/kaggle/input/inputs/textstat-0.7.3-py3-none-any.whl
/kaggle/input/inputs/vectorizer.pk
/kaggle/input/inputs/pyphen-0.15.0-py3-none-any.whl
/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/2/config.json
/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/2/tokenizer.json
/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/2/metadata.json
/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/2/model.weights.h5
/kaggle/input/deberta_v3/keras/deberta_v3_extra_small_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv


In [2]:
BASE_PATH = '/kaggle/input/learning-agency-lab-automated-essay-scoring-2'

In [3]:
!pip install "/kaggle/input/inputs/pyphen-0.15.0-py3-none-any.whl"
!pip install "/kaggle/input/inputs/textstat-0.7.3-py3-none-any.whl"
!pip install "/kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl"

Processing /kaggle/input/inputs/pyphen-0.15.0-py3-none-any.whl
Processing /kaggle/input/inputs/textstat-0.7.3-py3-none-any.whl
Processing /kaggle/input/pyspellchecker/pyspellchecker-0.7.2-py3-none-any.whl


In [4]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [5]:
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from spellchecker import SpellChecker
from textstat import textstat
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import cohen_kappa_score, accuracy_score
from sklearn.model_selection import train_test_split  # Import package

import pickle # To save models

# Ensure you have the required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


/opt/conda/lib/python3.10/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

# Get existing models (RF + XGBoost)

In [6]:
def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = Cmatrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return 'kappa',(1.0 - numerator / denominator)


In [7]:
with open('/kaggle/input/random-forest/random_forest_model.pkl', 'rb') as rf_file:
    rf_model = pickle.load(rf_file)

with open('/kaggle/input/inputs/xgb_aes_model.pkl', 'rb') as xgb_file:
    xgb_model = pickle.load(xgb_file)

# LOAD NEURAL NETWORK

In [8]:
import os
os.environ["KERAS_BACKEND"] = "jax"  # "jax" or "tensorflow" or "torch"

import keras_nlp
import keras
import keras.backend as K
import tensorflow as tf
from keras.models import load_model

import matplotlib.pyplot as plt
import matplotlib as mpl

# Class for the evaluation metric which is the Weighted Kappa
# This metric is used for ordinal data
class WeightedKappa(keras.metrics.Metric):
    def __init__(self, num_classes=6, epsilon=1e-6):
        super().__init__(name="weighted_kappa")
        self.num_classes = num_classes
        self.epsilon = epsilon

        label_vec = keras.ops.arange(num_classes, dtype=keras.backend.floatx())
        self.row_label_vec = keras.ops.reshape(label_vec, [1, num_classes])
        self.col_label_vec = keras.ops.reshape(label_vec, [num_classes, 1])
        col_mat = keras.ops.tile(self.col_label_vec, [1, num_classes])
        row_mat = keras.ops.tile(self.row_label_vec, [num_classes, 1])
        self.weight_mat = (col_mat - row_mat) ** 2

        self.numerator = self.add_weight(name="numerator", initializer="zeros")
        self.denominator = self.add_weight(name="denominator", initializer="zeros")
        self.o_sum = self.add_weight(name = 'o_sum', initializer = 'zeros')
        self.e_sum = self.add_weight(name = 'e_sum', initializer = 'zeros')

    def update_state(self, y_true, y_pred, **args):
        # revert ordinal regression labels to classification labels
        y_true = keras.ops.one_hot(keras.ops.sum(y_true, axis=-1) - 1, 6)
        y_pred = keras.ops.one_hot(
            keras.ops.sum(keras.ops.cast(y_pred > 0.5, dtype="int8"), axis=-1) - 1, 6
        )
        # weighted kappa calculation
        y_true = keras.ops.cast(y_true, dtype=self.col_label_vec.dtype)
        y_pred = keras.ops.cast(y_pred, dtype=self.weight_mat.dtype)
        batch_size = keras.ops.shape(y_true)[0]

        cat_labels = keras.ops.matmul(y_true, self.col_label_vec)
        cat_label_mat = keras.ops.tile(cat_labels, [1, self.num_classes])
        row_label_mat = keras.ops.tile(self.row_label_vec, [batch_size, 1])

        weight = (cat_label_mat - row_label_mat) ** 2

        self.numerator.assign_add(keras.ops.sum(weight * y_pred))
        label_dist = keras.ops.sum(y_true, axis=0, keepdims=True)
        pred_dist = keras.ops.sum(y_pred, axis=0, keepdims=True)
        w_pred_dist = keras.ops.matmul(
            self.weight_mat, keras.ops.transpose(pred_dist, [1, 0])
        )
        self.denominator.assign_add(
            keras.ops.sum(keras.ops.matmul(label_dist, w_pred_dist))
        )

        self.o_sum.assign_add(keras.ops.sum(y_pred))
        self.e_sum.assign_add(keras.ops.sum(
            keras.ops.matmul(keras.ops.transpose(label_dist, [1, 0]), pred_dist)
        ))

    def result(self):
        return 1.0 - (
            keras.ops.divide_no_nan(self.numerator, self.denominator)
            * keras.ops.divide_no_nan(self.e_sum, self.o_sum)
        )

    def reset_state(self):
        self.numerator.assign(0)
        self.denominator.assign(0)
        self.o_sum.assign(0)
        self.e_sum.assign(0)

print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("KerasNLP:", keras_nlp.__version__)

class CFG:
    seed = 42  # Random seed
    preset = "deberta_v3_extra_small_en" # Name of pretrained models
    sequence_length = 512  # Input sequence length
    epochs = 5 # Training epochs
    batch_size = 32  # Batch size
    scheduler = 'cosine'  # Learning rate scheduler

keras.utils.set_random_seed(CFG.seed)

2024-05-21 22:02:04.722150: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 22:02:04.722247: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 22:02:04.853808: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow: 2.15.0
Keras: 3.2.1
KerasNLP: 0.9.3


In [9]:
# LOAD NEURAL NETWORK

preprocessor = keras_nlp.models.DebertaV3Preprocessor.from_preset(
    preset=CFG.preset, # Name of the model
    sequence_length=CFG.sequence_length, # Max sequence length, will be padded if shorter
)

def keras_pipeline():
    print("----------Reading DF----------")
    df = pd.read_csv(f'{BASE_PATH}/train.csv')  # Read CSV file into a DataFrame

    train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df["score"])

    print("----------Transforming Labels----------")
    train_df["label"] = to_ordinal(train_df.score.values).tolist()
    valid_df["label"] = to_ordinal(valid_df.score.values).tolist()
    print("----------Done----------")
    print("----------Preprocessing Data and Building Train and Valid Sets----------")
    # Train Data
    train_texts = train_df.full_text.tolist()  # Extract training texts
    train_labels = np.array(train_df.label.tolist())  # Extract training labels

    # Build training dataset
    train_ds = build_dataset(
        train_texts, train_labels, batch_size=CFG.batch_size, shuffle=True
    )

    # Valid Data
    valid_texts = valid_df.full_text.tolist()  # Extract validation texts
    valid_labels = np.array(valid_df.label.tolist())  # Extract validation labels

    # Build validation dataset
    valid_ds = build_dataset(
        valid_texts, valid_labels, batch_size=CFG.batch_size, shuffle=False
    )
    print("----------Done----------")
    print("----------Set Learning Rate callback to adjust the LR as the model learns----------")
    lr_cb = get_lr_callback(CFG.batch_size, plot=False)
    print("----------Done----------")
    print("----------Save a checkpoint of the best model performance during training----------")
    ckpt_cb = keras.callbacks.ModelCheckpoint(
    "best_model.weights.h5",
    monitor="val_weighted_kappa",
    save_best_only=True,
    save_weights_only=True,
    mode="max",
    )
    print("----------Done----------")
    print("----------Create the model----------")
    # Create a DebertaV3Classifier model
    classifier = keras_nlp.models.DebertaV3Classifier.from_preset(
        CFG.preset, preprocessor=None, num_classes=6
    )
    inputs = classifier.input
    logits = classifier(inputs)

    # Compute final output
    outputs = keras.layers.Activation("sigmoid")(logits)

    # Build Model
    model = keras.Model(inputs, outputs)

    # Compile the model with optimizer, loss, and metrics
    model.compile(
        optimizer=keras.optimizers.Adam(5e-6),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=[
            WeightedKappa()
        ],
    )
    model.summary()

    print("----------Done----------")
    print("----------Train the model----------")
    # The model is being trained during 5 epochs (amount of passes through the data)
    # The epoch is set to 5, having a greater than could increase predictive power
    history = model.fit(
        train_ds,
        epochs=CFG.epochs,
        validation_data=valid_ds,
        callbacks=[lr_cb, ckpt_cb]
    )
    print("----------Done----------")
    print("----------Find epoch with best predictive power----------")
    # Find the epoch with the best validation accuracy
    best_epoch = np.argmax(model.history.history['val_weighted_kappa'])
    best_score = model.history.history['val_weighted_kappa'][best_epoch]
    best_loss = model.history.history['val_loss'][best_epoch]

    print("----------Done----------")
    print("----------Generate predictions----------")

     # Load best checkpoint
    model.load_weights("best_model.weights.h5")

    # Do inference
    train_preds = model.predict(train_ds, verbose=1)

    # Convert probabilities to class labels
    train_preds = np.sum((train_preds>0.5).astype(int), axis=-1).clip(1, 6)

    print("----------Done----------")
    print("----------Save model and predictions----------")

    return train_preds, model, train_df, valid_df




# Function to build the dataset ( apply preprocessing, etc)
def build_dataset(texts, labels=None, batch_size=32,
                  cache=True, drop_remainder=True,
                  shuffle=1024):
    AUTO = tf.data.AUTOTUNE  # AUTOTUNE option
    slices = (texts,) if labels is None else (texts, labels)  # Create slices
    ds = tf.data.Dataset.from_tensor_slices(slices)  # Create dataset from slices
    ds = ds.cache() if cache else ds  # Cache dataset if enabled
    ds = ds.map(preprocess_fn, num_parallel_calls=AUTO)  # Map preprocessing function
    opt = tf.data.Options()  # Create dataset options
    if shuffle:
        ds = ds.shuffle(shuffle, seed=CFG.seed)  # Shuffle dataset if enabled
        opt.experimental_deterministic = False
    ds = ds.with_options(opt)  # Set dataset options
    ds = ds.batch(batch_size, drop_remainder=drop_remainder)  # Batch dataset
    ds = ds.prefetch(AUTO)  # Prefetch next batch
    return ds


# Helper func to preprocess data
def preprocess_fn(text, label=None):
    text = preprocessor(text)  # Preprocess text
    return (text, label) if label is not None else text  # Return processed text and label if available




# Helper functional to convert the label into ordinal
def to_ordinal(y, num_classes=None, dtype="float32"):
    """Converts a class vector (integers) to an ordinal regression matrix.

    This utility encodes class vector to ordinal regression/classification
    matrix where each sample is indicated by a row and rank of that sample is
    indicated by number of ones in that row.

    Args:
        y: Array-like with class values to be converted into a matrix
            (integers from 0 to `num_classes - 1`).
        num_classes: Total number of classes. If `None`, this would be inferred
            as `max(y) + 1`.
        dtype: The data type expected by the input. Default: `'float32'`.

    Returns:
        An ordinal regression matrix representation of the input as a NumPy
        array. The class axis is placed last.
    """
    y = np.array(y, dtype="int")
    input_shape = y.shape

    # Shrink the last dimension if the shape is (..., 1).
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])

    y = y.reshape(-1)
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    range_values = np.arange(num_classes - 1)
    range_values = np.tile(np.expand_dims(range_values, 0), [n, 1])
    ordinal = np.zeros((n, num_classes - 1), dtype=dtype)
    ordinal[range_values < np.expand_dims(y, -1)] = 1
    output_shape = input_shape + (num_classes - 1,)
    ordinal = np.reshape(ordinal, output_shape)
    return ordinal

import math

# Update the learning rate
def get_lr_callback(batch_size=8, mode='cos', epochs=10, plot=False):
    lr_start, lr_max, lr_min = 0.6e-5, 0.3e-5 * batch_size, 0.3e-5
    lr_ramp_ep, lr_sus_ep, lr_decay = 2, 0, 0.75

    def lrfn(epoch):  # Learning rate update function
        if epoch < lr_ramp_ep: lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
        elif epoch < lr_ramp_ep + lr_sus_ep: lr = lr_max
        elif mode == 'exp': lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
        elif mode == 'step': lr = lr_max * lr_decay**((epoch - lr_ramp_ep - lr_sus_ep) // 2)
        elif mode == 'cos':
            decay_total_epochs, decay_epoch_index = epochs - lr_ramp_ep - lr_sus_ep + 3, epoch - lr_ramp_ep - lr_sus_ep
            phase = math.pi * decay_epoch_index / decay_total_epochs
            lr = (lr_max - lr_min) * 0.5 * (1 + math.cos(phase)) + lr_min
        return lr

    if plot:  # Plot lr curve if plot is True
        plt.figure(figsize=(10, 5))
        plt.plot(np.arange(epochs), [lrfn(epoch) for epoch in np.arange(epochs)], marker='o')
        plt.xlabel('epoch'); plt.ylabel('lr')
        plt.title('LR Scheduler')
        plt.show()

    return keras.callbacks.LearningRateScheduler(lrfn, verbose=False)  # Create lr callback



Attaching 'tokenizer.json' from model 'keras/deberta_v3/keras/deberta_v3_extra_small_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/deberta_v3/keras/deberta_v3_extra_small_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/deberta_v3/keras/deberta_v3_extra_small_en/2' to your Kaggle notebook...


In [10]:
df = pd.read_csv(f'{BASE_PATH}/train.csv')  # Read CSV file into a DataFrame
train_df, valid_df = train_test_split(df, test_size=0.2, stratify=df["score"])

# Stacking

In [11]:
import pickle

def extra_feature_pipeline(df):

    print("---------CALCULATING ESSAY LENGTHS---------")
    train_df = calculate_essay_lengths(df, "full_text")
    print("---------FINISHED CALCULATING ESSAY LENGTHS---------\n")

    print("---------ANALYZING SENTIMENTS---------")
    train_df = analyze_sentiment(df, "full_text")
    print("---------FINISHED ANALYZING SENTIMENTS---------\n")

    print("---------ANALYZING READABILITY---------")
    readability_train_df = analyze_readability(df, "full_text")
    train_df["fkg_score"] = readability_train_df["Flesch-Kincaid"]
    train_df["gf_score"] = readability_train_df["Gunning Fog"]
    print("---------FINISHED ANALYZING READABILITY---------\n")

    print("---------ANALYZING LEXICAL DIVERSITY AND SPELLING MISTAKES---------")
    train_df = lexical_diversity_and_mistakes(df, "full_text")
    print("---------FINISHED ANALYZING LEXICAL DIVERSITY AND SPELLING MISTAKES---------\n")

    print("---------GETTING DIFFICULT WORD COUNT---------")
    train_df = get_difficult_word_count(df, "full_text")
    print("---------FINISHED GETTING DIFFICULT WORD COUNT---------\n")

    print("---------STARTING PREPROCESSING---------")
    train_df["full_text"] = train_df["full_text"].apply(preprocess_text)
    print("---------FINISHED PREPROCESSING---------\n")

    print("---------STARTING VECTORIZATION---------")
    with open('/kaggle/input/inputs/vectorizer.pk', 'rb') as file:
        vectorizer = pickle.load(file)
    train_df_vectorized = add_tfidf_features(df, "full_text", vectorizer)
    print("---------FINISHED VECTORIZATION---------\n")

    return train_df, train_df_vectorized

def add_tfidf_features(train_df,text_column, vectorizer):
    train_tfidf = vectorizer.transform(train_df[text_column])

    train_dense = train_tfidf.toarray()

    train_tfidf_df = pd.DataFrame(train_dense, columns=[f'tfid_{i}' for i in range(train_dense.shape[1])])

    train_tfidf_df['essay_id'] = train_df['essay_id'].values

    train_df = train_df.merge(train_tfidf_df, on='essay_id', how='left')

    return train_df

# Helper Functions

def calculate_essay_lengths(df, text_column):
    df['char_essay_length'] = df[text_column].apply(len)
    df['words_essay_length'] = df[text_column].apply(lambda x: len(x.split()))
    df['sentence_essay_length'] = df[text_column].apply(lambda x: len(x.split('.')))
    return df

def analyze_sentiment(df, text_column):
    # Initialize the sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    # Analyze the sentiment of each essay
    sentiment_scores = df[text_column].apply(lambda x: sid.polarity_scores(x)['compound'])

    # Add sentiment scores to the dataframe
    df['sentiment_score'] = sentiment_scores

    # Overall sentiment analysis
    positive_count = sum(sentiment_scores > 0)
    negative_count = sum(sentiment_scores < 0)
    neutral_count = len(sentiment_scores) - positive_count - negative_count

    # Data for visualization
    categories = ['Positive', 'Negative', 'Neutral']
    counts = [positive_count, negative_count, neutral_count]


    return df



def analyze_readability(df, text_column):
    # Analyzing readability scores for each essay
    readability_scores = []
    for essay in df[text_column]:
        # Compute Flesch-Kincaid Grade Level
        fkg_score = textstat.flesch_kincaid_grade(essay)

        # Compute Gunning Fog Index
        gunning_fog_score = textstat.gunning_fog(essay)

        # Add scores to list
        readability_scores.append({'Flesch-Kincaid': fkg_score, 'Gunning Fog': gunning_fog_score})

    # Creating a dataframe to store the scores
    readability_df = pd.DataFrame(readability_scores)

    return readability_df

# Inspo from https://www.kaggle.com/code/kuangank/ase-fighting
def lexical_diversity_and_mistakes(df, text_column):
    spell_checker = SpellChecker()

    lexical_diversities = []
    spelling_mistake_counts = []
    spelling_mistake_ratios = []

    for text in df[text_column]:
        tokens = word_tokenize(text)
        unique_tokens = set(tokens)

        # Calculate lexical diversity
        if len(tokens) == 0:
            lexical_diversity = 0
        else:
            lexical_diversity = len(unique_tokens) / len(tokens)

        # Calculate spelling mistakes
        spelling_mistake_count = len(spell_checker.unknown(token for token in tokens if token.isalpha()))

        # Calculate spelling mistake ratio
        if len(tokens) == 0:
            spelling_mistake_ratio = 0
        else:
            spelling_mistake_ratio = spelling_mistake_count / len(tokens)

        lexical_diversities.append(lexical_diversity)
        spelling_mistake_counts.append(spelling_mistake_count)
        spelling_mistake_ratios.append(spelling_mistake_ratio)

    df['lexical_diversity'] = lexical_diversities
    df['spelling_mistake_count'] = spelling_mistake_counts
    df['spelling_mistake_ratio'] = spelling_mistake_ratios

    return df

def get_difficult_word_count(df, text_column):
    difficult_word_counts = []
    difficult_word_ratios = []

    for text in df[text_column]:
        tokens = word_tokenize(text)

        # Calculate difficult words
        difficult_word_count = textstat.difficult_words(text)

        # Calculate difficult word ratio
        if len(tokens) == 0:
            difficult_word_ratio = 0
        else:
            difficult_word_ratio = difficult_word_count / len(tokens)

        difficult_word_counts.append(difficult_word_count)
        difficult_word_ratios.append(difficult_word_ratio)

    df['difficult_words'] = difficult_word_counts
    df['difficult_word_ratio'] = difficult_word_ratios

    return df

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove punctuation and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)


In [12]:
train_df_extra, train_df_vect = extra_feature_pipeline(train_df)

---------CALCULATING ESSAY LENGTHS---------
---------FINISHED CALCULATING ESSAY LENGTHS---------

---------ANALYZING SENTIMENTS---------
---------FINISHED ANALYZING SENTIMENTS---------

---------ANALYZING READABILITY---------
---------FINISHED ANALYZING READABILITY---------

---------ANALYZING LEXICAL DIVERSITY AND SPELLING MISTAKES---------
---------FINISHED ANALYZING LEXICAL DIVERSITY AND SPELLING MISTAKES---------

---------GETTING DIFFICULT WORD COUNT---------
---------FINISHED GETTING DIFFICULT WORD COUNT---------

---------STARTING PREPROCESSING---------
---------FINISHED PREPROCESSING---------

---------STARTING VECTORIZATION---------
---------FINISHED VECTORIZATION---------



In [13]:
valid_df_extra, valid_df_vect = extra_feature_pipeline(valid_df)

---------CALCULATING ESSAY LENGTHS---------
---------FINISHED CALCULATING ESSAY LENGTHS---------

---------ANALYZING SENTIMENTS---------
---------FINISHED ANALYZING SENTIMENTS---------

---------ANALYZING READABILITY---------
---------FINISHED ANALYZING READABILITY---------

---------ANALYZING LEXICAL DIVERSITY AND SPELLING MISTAKES---------
---------FINISHED ANALYZING LEXICAL DIVERSITY AND SPELLING MISTAKES---------

---------GETTING DIFFICULT WORD COUNT---------
---------FINISHED GETTING DIFFICULT WORD COUNT---------

---------STARTING PREPROCESSING---------
---------FINISHED PREPROCESSING---------

---------STARTING VECTORIZATION---------
---------FINISHED VECTORIZATION---------



In [14]:
# Valid Data
# valid_texts = valid_df.full_text.tolist()  # Extract validation texts
# valid_labels = np.array(valid_df.label.tolist())  # Extract validation labels

# # Build validation dataset
# valid_ds = build_dataset(
#         valid_texts, valid_labels, batch_size=CFG.batch_size, shuffle=False
#     )

In [15]:
valid_df_e = valid_df_extra.drop(["essay_id","full_text"], axis = 1)
valid_df_v = valid_df_vect.drop(["essay_id","full_text"], axis = 1)

In [16]:
# Helper function to convert TensorFlow dataset to NumPy array
def tf_dataset_to_numpy(tf_dataset):
    # Accumulate batches into a list
    batches = [batch for batch in tf_dataset]
    # If the dataset has labels, unpack the tuples
    if isinstance(batches[0], tuple):
        # Assume (inputs, labels)
        inputs, labels = zip(*batches)
        inputs_np = np.concatenate(inputs, axis=0)
        labels_np = np.concatenate(labels, axis=0)
        return inputs_np, labels_np
    else:
        inputs_np = np.concatenate(batches, axis=0)
        return inputs_np

In [17]:
# Handle NaNs by dropping rows with any NaNs in valid_df_e and valid_df_v
valid_df_e = valid_df_e.dropna().reset_index(drop=True)
valid_df_v = valid_df_v.dropna().reset_index(drop=True)

y_test = valid_df_e['score']

valid_df_e = valid_df_e.drop(["score"], axis = 1)
valid_df_v = valid_df_v.drop(["score"], axis = 1)

# Generate predictions from the base models
# nn_predictions = model.predict(valid_ds, verbose=1)
rf_predictions = rf_model.predict(valid_df_e)
xgb_predictions = xgb_model.predict(valid_df_v)

# Ensure the predictions are in the correct shape
# nn_predictions = nn_predictions.reshape(-1, 1)
rf_predictions = rf_predictions.reshape(-1, 1)
xgb_predictions = xgb_predictions.reshape(-1, 1)

# Output predictions for manual joining
# nn_predictions_df = pd.DataFrame(nn_predictions, columns=['nn_predictions'])
rf_predictions_df = pd.DataFrame(rf_predictions, columns=['rf_predictions'])
xgb_predictions_df = pd.DataFrame(xgb_predictions, columns=['xgb_predictions'])

# Print the predictions DataFrames
# print("NN Predictions:")
# print(nn_predictions_df.head())

print("RF Predictions:")
print(rf_predictions_df.head())

print("XGB Predictions:")
print(xgb_predictions_df.head())

RF Predictions:
   rf_predictions
0               3
1               3
2               4
3               6
4               3
XGB Predictions:
   xgb_predictions
0                2
1                2
2                3
3                5
4                2


In [18]:
merged_df = pd.concat([rf_predictions_df, xgb_predictions_df, valid_df_e], axis=1)

In [19]:
from sklearn.svm import SVC
classifier = SVC()

In [20]:
classifier.fit(merged_df, y_test)

SVC()

In [21]:
# Assuming nn_predictions_df is a DataFrame containing the probabilities
# Ensure nn_predictions_df is a DataFrame
# nn_predictions_df = pd.DataFrame(nn_predictions_df)

# # Apply the linear transformation to map probabilities from [0, 1] to [1, 6]
# nn_mapped_preds = 1 + nn_predictions_df * 5

# # If you need to convert this to integers, you can round or use another method
# nn_mapped_preds = nn_mapped_preds.round().astype(int)

# # Convert the result to a DataFrame if needed
# nn_mapped_preds_df = pd.DataFrame(nn_mapped_preds)

# # Print the first few rows to verify
# print(nn_mapped_preds_df.head())

# # Count the number of rows where the mapped value is 1
# count_ones = nn_mapped_preds_df[nn_mapped_preds_df == 1].count().sum()
# print("Count of rows with value 1:", count_ones)


In [22]:
train = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv")
test = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv")
submission = pd.read_csv("/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv")

In [23]:
test_df_rf, test_df_vectorized = extra_feature_pipeline(test)

test_df_rf = test_df_rf.dropna()
test_df_vectorized = test_df_vectorized.dropna()

test_df_rf = test_df_rf.drop(["essay_id","full_text"],axis=1)
test_df_vectorized = test_df_vectorized.drop(["essay_id","full_text"],axis=1)


# Generate predictions from the base models
# nn_predictions = model.predict(valid_ds, verbose=1)
rf_predictions_final = rf_model.predict(test_df_rf)
xgb_predictions = xgb_model.predict(test_df_vectorized)

# Ensure the predictions are in the correct shape
# nn_predictions = nn_predictions.reshape(-1, 1)
rf_predictions_final = rf_predictions_final.reshape(-1, 1)
xgb_predictions = xgb_predictions.reshape(-1, 1)

# Output predictions for manual joining
# nn_predictions_df = pd.DataFrame(nn_predictions, columns=['nn_predictions'])
rf_predictions_df_final = pd.DataFrame(rf_predictions_final, columns=['rf_predictions'])
xgb_predictions_df_final = pd.DataFrame(xgb_predictions, columns=['xgb_predictions'])

---------CALCULATING ESSAY LENGTHS---------
---------FINISHED CALCULATING ESSAY LENGTHS---------

---------ANALYZING SENTIMENTS---------
---------FINISHED ANALYZING SENTIMENTS---------

---------ANALYZING READABILITY---------
---------FINISHED ANALYZING READABILITY---------

---------ANALYZING LEXICAL DIVERSITY AND SPELLING MISTAKES---------
---------FINISHED ANALYZING LEXICAL DIVERSITY AND SPELLING MISTAKES---------

---------GETTING DIFFICULT WORD COUNT---------
---------FINISHED GETTING DIFFICULT WORD COUNT---------

---------STARTING PREPROCESSING---------
---------FINISHED PREPROCESSING---------

---------STARTING VECTORIZATION---------
---------FINISHED VECTORIZATION---------



In [24]:
final_preds = pd.concat([rf_predictions_df_final, xgb_predictions_df_final, test_df_rf], axis=1)
final_preds

,rf_predictions,xgb_predictions,char_essay_length,words_essay_length,sentence_essay_length,sentiment_score,fkg_score,gf_score,lexical_diversity,spelling_mistake_count,spelling_mistake_ratio,difficult_words,difficult_word_ratio
0,3,2,2677,498,14,0.9937,14.7,17.33,0.455046,16,0.029358,60,0.110092
1,3,2,1669,332,20,0.7705,5.4,7.48,0.452830,7,0.018868,24,0.064690
2,4,3,3077,550,25,-0.9731,9.9,11.49,0.401653,7,0.011570,67,0.110744


In [25]:
# classifier.fit(final_preds, y_test)
val_pred = classifier.predict(final_preds)

In [26]:
val_pred

array([4, 3, 4])

In [27]:
submission['score'] = classifier.predict(final_preds)

In [28]:
submission.to_csv("submission.csv",index = False)